In [1]:
from data import FundamentalDataset, PriceDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import datetime as dt
import itertools
from utils import Defaults


DEFAULTS = Defaults
fund_data = FundamentalDataset()
price_data = PriceDataset()

In [2]:
from torch.utils.data import DataLoader, Dataset

In [3]:
def collate_fn(batch):
    return batch

In [4]:
# shuffle data for training combined models
from random import shuffle

batch_size = 16
training_periods = price_data.period_list.period.iloc[:-price_data.get_len()[-1] // 3]
shuffle(training_periods)

In [4]:
class SingleDataloader(DataLoader):
    def __init__(
            self, 
            dataset: Dataset, 
            batch_size: int=16, 
            **kwargs):
        super().__init__(dataset=dataset, batch_size=batch_size, **kwargs)
        self.index= list(itertools.product(*dataset.index))
    def _next_index(self):
        for next_item in self.index:
            yield next_item
    def _next_data(self):
        for index in self._next_index():
            print(index)
            data = self.dataset[index]
            yield data


In [6]:
fundamental_data_loader = DataLoader(
    fund_data,
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=16
)

c:\Users\p.peng\Anaconda3\envs\stockencoder_env\lib\site-packages\torch\utils\data\dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
next(fundamental_data_loader.__iter__())

In [53]:
import pandas as pd


datetime.date(1997, 1, 1)

In [9]:
len(fund_data.index)

2

In [ ]:
from models.autoencoder import BaseAutoEncoder

model = BaseAutoEncoder(encoding_dim=3, num_transformer_layers=[1, 1], dims=)

In [29]:
encoder_layer = nn.TransformerEncoderLayer(1, 1, batch_first=True)
encoder = nn.TransformerEncoder(encoder_layer, 1)

encoder(input)

tensor([[[0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.]]], grad_fn=<NativeLayerNormBackward0>)

In [3]:
price_data[(1, dt.date(2022, 1, 1))]

c:\Users\p.peng\Anaconda3\envs\stockencoder_env\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


tensor([[-1.0000e+10,  2.2513e-01, -1.0000e+10,  8.9167e-01,  9.5669e-01],
        [-1.0000e+10,  2.2591e-01, -1.0000e+10,  8.9041e-01,  9.5619e-01],
        [-1.0000e+10,  2.2721e-01, -1.0000e+10,  8.8854e-01,  9.5544e-01],
        ...,
        [-1.0000e+10,  2.7621e-01, -6.6862e+00,  8.8338e-01,  9.2140e-01],
        [-1.0000e+10,  2.7616e-01, -2.5919e+00,  8.8354e-01,  9.2150e-01],
        [-1.0000e+10,  2.7332e-01, -1.0000e+10,  8.9210e-01,  9.2715e-01]])